In [1]:
'''
Scroll down to find random distribution + TODO work of propensities
'''

import networkx as nx
import random
import copy
import time
import matplotlib.pyplot as plt
import sys
%matplotlib inline

def progressBar(value, endvalue, bar_length=20):

        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))

        sys.stdout.write("\rCalculating: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
        sys.stdout.flush()

        
def kawai_draw(G, infected=[]) :
    colors = []
    for node in G :
        if node in infected :
            colors.append('red')
        else :
            colors.append('green')
    nx.draw(G, layout=nx.kamada_kawai_layout(G), node_color=colors, node_size = 30, alpha = 0.7)
    plt.show()
    
G = nx.scale_free_graph(1000)
G = G.to_undirected()

In [2]:
#kawai_draw(G)

In [3]:
def generate_egos(G, node, max_radius):
    resulting_G = []
    
    G_ego = nx.ego_graph(G, node, radius=1)
    if len(G_ego) > 1 :
        resulting_G.append(G_ego)
        G_ego_previous = G_ego

        for i in range(2, max_radius+1) :
            G_ego = nx.ego_graph(G, node, radius=i)
            G_ego_toremove = copy.deepcopy(G_ego)
            G_ego_toremove.remove_nodes_from(G_ego_previous)
            G_ego_previous = G_ego

            if len(G_ego_toremove) > 0 :
                resulting_G.append(G_ego_toremove)
            else :
                break

        resulting_G[0].remove_node(node)
    return resulting_G

def generate_egos_allnodes(G, max_radius) :
    result = []
    
    nodecount = 1
    for node in G :
        progressBar(nodecount, len(G), 50)
        result.append(generate_egos(G, node, max_radius))
        nodecount +=1
        
    return result

radius_split = generate_egos_allnodes(G, 10)

Calculating: [------------------------------------------------->] 100%

In [4]:
def infected_percentage(G, infected) :
    
    count = 0
    
    for node in G :
        if node in infected :
            count += 1
        
    return count / len(G)

def infected_average_by_radius(G, Gs_list, radius, infected) :
    node_count = []
    percentage_sum = []
    for _ in range(0, radius) :
        node_count.append(0)
        percentage_sum.append(0)
            
    current_node = 0
    for node in Gs_list :
        if current_node in infected :
            current_radius = 0
            for item in node :
                percentage_sum[current_radius] += infected_percentage(item, infected)
                node_count[current_radius] += 1
                current_radius += 1
        current_node += 1
        
    average_by_radius = []
    for i in range(0, radius) :
        if node_count[i] > 0 :
            average_by_radius.append(percentage_sum[i] / node_count[i])
        
    return average_by_radius

In [5]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.SIModel as si

# Model selection
model = si.SIModel(G)

# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.01) # prob of infecteded
cfg.add_model_parameter("percentage_infected", 0.5) # initially infected  %
model.set_initial_status(cfg)

# Simulation execution
iterations = model.iteration_bunch(60)

In [6]:
infected = []
infected_count = [] # added
avg_traits = [] # added
for it in iterations :
    for node, i in it['status'].items() :
        if i == 1:
            infected.append(node)
            
    print('Infected count', len(infected))    
    print(infected_average_by_radius(G, radius_split, 10, infected))
    
    avg_traits.append(infected_average_by_radius(G, radius_split, 10, infected)) # needed for propensity
    infected_count.append(len(infected)) # needed for the random graph generation

Infected count 500
[0.5241867688383167, 0.4868177417935251, 0.5024602310467817, 0.5298758326966216, 0.29851049194393753, 0.09772702063589243, 0.0]
Infected count 510
[0.7469365146377531, 0.5391136965333985, 0.5155538273781716, 0.5320713530679261, 0.2954276096186038, 0.09664517169896374, 0.0]
Infected count 521
[0.7750173011552415, 0.5633548417493335, 0.5265002784994891, 0.5335945922915845, 0.2940063919395734, 0.0952394237469788, 0.0]
Infected count 531
[0.7836423163737233, 0.5773930855332101, 0.5343715716550156, 0.5428705870100637, 0.2939778873268492, 0.0971894816857243, 0.0]
Infected count 537
[0.8026276004185233, 0.5854993959724297, 0.5419483194676332, 0.5453249317820973, 0.2934560933337765, 0.0961481658105201, 0.0]
Infected count 546
[0.8094707033277414, 0.595268543689002, 0.5507907422867211, 0.547737323536236, 0.29292133379890417, 0.09479396629206208, 0.0]
Infected count 551
[0.8215770482899244, 0.6028163516071691, 0.555915608043797, 0.5489494060026389, 0.2923327234609084, 0.094131

Infected count 760
[0.9683382982095081, 0.8318166336972715, 0.7625251675650674, 0.6983954307107411, 0.6803586241851984, 0.5556166669332315, 0.5232804232804233, 0.75]
Infected count 760
[0.9683382982095081, 0.8318166336972715, 0.7625251675650674, 0.6983954307107411, 0.6803586241851984, 0.5556166669332315, 0.5232804232804233, 0.75]
Infected count 762
[0.9687667513569068, 0.8328822127049085, 0.765690038393056, 0.7009373871636096, 0.6810486783288788, 0.5558010229357985, 0.5232804232804233, 0.75]
Infected count 765
[0.9697522796665916, 0.8369766277507776, 0.770233511600183, 0.7019414878974295, 0.6816998648517999, 0.5553959882817252, 0.5232804232804233, 0.75]
Infected count 766
[0.970956463195415, 0.8373943295169317, 0.7719050740529436, 0.702730018421313, 0.6817666434157482, 0.5552623370482652, 0.5232804232804233, 0.75]
Infected count 769
[0.9711414676882955, 0.8395353830618338, 0.77593357642256, 0.7040216702242044, 0.6821864535868469, 0.5548653928848892, 0.5232804232804233, 0.75]
Infected c

In [7]:
#kawai_draw(G, infected)

In [ ]:
''' Start Random distribution of infecteds section '''
import random
from collections import defaultdict
from copy import deepcopy # only used for testing
 
# auxiliary: print elements different between 2 lists
def print_list_differences(prev_list, curr_list):
    s = set(prev_list) # transform to set, because O(1) to find elem
    temp = [x for x in curr_list if x not in s]
    print(temp)

num_nodes = 1000
G_rand = nx.scale_free_graph(num_nodes).to_undirected() # this will be our random graph
radius_split_rand = generate_egos_allnodes(G_rand, 10)

Calculating: [>                                                 ] 1%

In [ ]:
def simulate_random_infecteds(G_rand, num_nodes, radius_split_rand, infected_count):
    '''
    Randomly picks the same number of infecteds as in the 
    non-random model (for a each iteration of the spreading)
    Returns a list of @infected_average_by_radius lists
    '''
    print(infected_count)
    random_infected = []
    random_not_infected = list(range(0, num_nodes))
    prev_num_infects = 0
    new_num_infects = 0
    avg_trait_sharing_by_radius = []
    for num_infects in infected_count:
        print("num_infects: ", num_infects)
        new_num_infects = num_infects - prev_num_infects
        print("new infecteds: ", new_num_infects)
        # prev_random_infected = deepcopy(random_infected) # for testing purposes

        for _ in range(new_num_infects):
            new_inf = random.choice(random_not_infected)
            random_infected.append(new_inf)
            random_not_infected.remove(new_inf)
            
        prev_num_infects = num_infects
        avg_trait_sharing_by_radius.append(infected_average_by_radius(G_rand, radius_split_rand, 10, random_infected))
        print(infected_average_by_radius(G_rand, radius_split_rand, 10, random_infected))    
        
        # print("¨¨¨¨¨¨¨¨¨Infected list: ", len(random_infected), " nodes\n", random_infected)
        # print("¨¨¨¨¨¨¨¨¨NOT Infected list:", len(random_not_infected), " nodes\n", random_not_infected) 
        # print_list_differences(prev_random_infected, random_infected)
    return (avg_trait_sharing_by_radius, random_infected)

avg_random_trait_sharing_by_radius, final_random_infected = simulate_random_infecteds(G_rand, num_nodes, radius_split_rand, infected_count)

In [ ]:
#kawai_draw(G_rand, final_random_infected)

In [ ]:
''' Unfinished '''
def propensity_by_radius(iteration, radius, avg_traits, rand_avg_traits):
    return (avg_traits[iteration-1][radius-1] / rand_avg_traits[iteration-1][radius-1]) - 1

def all_propensities(iteration, max_radius, avg_traits, rand_avg_traits):
    all_propensities = []
    for r in range(max_radius):
        all_propensities.append(propensity_by_radius(iteration, r, avg_traits, rand_avg_traits))
    return all_propensities

prop_one = propensity_by_radius(10, 1, avg_traits, avg_random_trait_sharing_by_radius)
prop_two = propensity_by_radius(10, 2, avg_traits, avg_random_trait_sharing_by_radius)
print(prop_one)

all_props = all_propensities(10, 6, avg_traits, avg_random_trait_sharing_by_radius)

# def normalized_propensities(max_radius, all_propensities):
#     normalizeds_propensisites = []
#     for r in max_radius:
        